# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 05: Batch Predictions</span>


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/eletricity/5_batch_predictions.ipynb)

## 🗒️ In this notebook you will see how to use deployed model for Batch prediction using  Hopsworks Feature Store : 

1. Loading batch data.
2. Precition using model from Model Registry.

![tutorial-flow](../../images/03_model.png)

### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [ ]:
!pip install -U hopsworks --quiet
!pip install -U tensorflow --quiet

In [ ]:
import tensorflow as tf

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---

## <span style="color:#ff5f27;"> ✨ Load recent batch data</span>

First, you will need to fetch the training dataset that you created in the previous notebook. You will use January-February data.

In [ ]:
import datetime
start_date = (datetime.datetime.now() - datetime.timedelta(days=4)) 
start_time = int(start_date.timestamp()) * 1000

In [ ]:
feature_view = fs.get_feature_view("electricity_feature_view", 1)
feature_view.init_batch_scoring(training_dataset_version=1)
df = feature_view.get_batch_data(start_time = start_time)
df.sort_values(["timestamp"], inplace=True)

In [ ]:
df

In [ ]:
df = df.drop(["day", "timestamp"], axis=1)

In [ ]:
df

---

## <span style="color:#ff5f27;">🚀 Using the model to predict electricity prices</span>

### <span style='color:#ff5f27'>🗄 Retrieving model from Model Registry</span>

In [ ]:
mr = project.get_model_registry()
model = mr.get_model("electricity_price_prediction_model", version = 1)
model_dir = model.download()
loaded_model = tf.saved_model.load(model_dir)
serving_function = loaded_model.signatures["serving_default"]

### <span style='color:#ff5f27'>🤖 Making the predictions</span>

In [ ]:
prediction = serving_function(tf.constant(df.values.reshape(-1, 4, 29), tf.float32))['dense'].numpy()#.flatten().tolist()
prediction

In [ ]:
se1_pred = []
se2_pred = []
se3_pred = []
se4_pred = []

for batch_n in range(len(prediction)):
    batch = prediction[batch_n]
    for window_n in range(4):
        se1_pred.append(batch[window_n][0])
        se2_pred.append(batch[window_n][1])
        se3_pred.append(batch[window_n][2])
        se4_pred.append(batch[window_n][3])

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

%config InlineBackend.figure_format='retina'
%matplotlib inline

plt.plot(se1_pred,color='red', label='SE1 price prediction')
plt.plot(se2_pred, color='blue', label='SE2 price prediction')
plt.plot(se3_pred, color='green', label='SE3 price prediction')
plt.plot(se4_pred, color='black', label='SE4 price prediction')
plt.xlabel('Time')
plt.ylabel('Price (scaled)')
plt.legend(loc='upper left')
fig = plt.gcf()
fig.set_size_inches(15, 5)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

plt.plot(se1_pred,color='red', label='SE1 price prediction')
plt.plot(se2_pred, color='blue', label='SE2 price prediction')
plt.plot(se3_pred, color='green', label='SE3 price prediction')
plt.plot(se4_pred, color='black', label='SE4 price prediction')
plt.xlabel('Time')
plt.ylabel('Price (scaled)')
plt.legend(loc='upper left')

#save figure to PNG file
plt.savefig('../../images/model_preds.png')